# Supervised Learning mit Features

Beim Supervised Learning auf Feature-Basis trainiert man ein Modell auf 
einer Liste von Merkmalen, die zuvor für den Text berechnet werden. So 
muss man zunächst entscheiden, welche Merkmale für das Training eines 
Modells interessant sein könnten, basierend auf der Zielsetzung des maschinellen Lernens. Hier ist ein Beispiel für ein Dokument im GermEval-Trainingskorpus:

Wir brauchen "relevance", um nur die relevanten Tweets für das 
Training auszuwählen. Den Wert in "sentiment" brauchen wir, weil er 
die Klassifikation enthält, die wir trainieren wollen. Den Text brauchen 
wir, weil wir daraus weitere Merkmale berechnen wollen. Ein solches 
Merkmal kann unser Wortlistenvergleich sein, der einen Polaritätswert 
als Ausgabe herausgibt. Weiterhin könnte die Anzahl der Negationen und 
Verstärker im Satz interessant sein. Der wichtigste Schritt ist, die Merkmale auszusuchen 
und dann zu berechnen. 

Hier ist eine Liste mit Negationen:


In [ ]:
negations = ['NIE', 'nicht', 'nich', 'kein', 'keine', 'Keine', 'ohne', 'nie', 'nein', 'keiner', 'nichts', 'weder', 'Weder', 'garnicht', 
'statt', 'Nix', 'nix', 'wäre','Wäre', ':-)', 'Gegensatz', 'kaum', 'Niemand']

Hier ist eine Liste mit Verstärkern:


In [ ]:
verstaerker = ['sehr', 'total', 'enorm', 'häufig', 'wirklich', 'völlig','voellig', 'absolut', 'rein', 'endlich', 'vollstes', 'viel', 'hoffen', 'genug', 'Ziemlich', 'scharf', 'ziemlich', 'kolossalen', 'kolossale', 'kolossales', 'stark', 'hohes', 'hohe', 'zusätzlichen', 
'lupenreinen', 'absolut', 'schleichend', 'definitiv']

Mit dieser Definition lassen sich die Negationen und Verstärker in einem 
tokenisierten Satz zählen:

In [ ]:
def neg_emp_in_sentence(sent):
    negs = 0
    emps = 0
    for tok in sent:
        if tok in negations:
            negs = negs +1
        elif tok in verstaerker:
            emps = emps +1
    return(negs, emps)

Die Dokumente aus dem GermEval-Trainingskorpus im XML-Format werden nun so vorbereitet, 
dass sie im TSV-Format mit dem Polaritätswert aus unserem 
Wortlistenvergleich, der Anzahl an Negationen und Verstärkern und der 
Annotation als positiv, negativ oder neutral stehen. Die nicht 
relevanten Dokumente werden aussortiert.

In [ ]:
from xml.etree import ElementTree as ET
from wortlistenvergleich import *
dev_data = open(r"germeval_2017_dev.xml","r",encoding="utf-8")
out = open("out.txt","w",encoding="utf-8")
tree = ET.parse(dev_data)
root = tree.getroot()

def convert_data():
    for document in root.iter('Document'):
        relevance = document.find('relevance').text
        sent = document.find('text').text
        sentiment = document.find('sentiment').text
        polarity = sentiment_analysis(sent)
        (negs, emps) = neg_emp_in_sentence(sent)
        if relevance == 'true':
            out.write(str(sent) + '\t' + str(polarity)
             + '\t' + str(negs) + '\t' + str(emps) + '\t' 
             + str(sentiment) + '\n')
    dev_data.close()
    out.close()
    
convert_data()

Ein Beispiel für eine Umwandlung in das neue Format ist:

Wir benennen die Datei um in germeval_trainingdata.txt.

Mit den so vorbereiteten Texten können wir jetzt ein Modell trainieren. 
Das hier vorgestellte Vorgehen orientiert sich an Jason Brownlee (https://machinelearningmastery.com/machine-learning-in-python-step-by-step/). 

Wir müssen einige Python-Module importieren, bevor wir beginnen: Pandas (https://pandas.pydata.org/about/) ist eine Bibliothek mit Werkzeugen für den Umgang mit Daten. Scikit-learn (https://scikit-learn.org/stable/), das wir als "sklearn" importieren, stellt Werkzeuge für das maschinelle Lernen bereit. Pickle (https://docs.python.org/3/library/pickle.html) benötigen wir, um das gelernte Modell abzuspeichern.



In [ ]:
import pandas

import sklearn

import pickle

from sklearn import model_selection

from sklearn.tree import DecisionTreeClassifier

Zunächst lokalisieren wir die vorbereitete Eingabedatei:

In [ ]:
input_data = (r"germeval_trainingdata.txt")

Dann verweisen wir auf die Bedeutung der Spalten:

In [ ]:
names = ['text', 'polarity', 'neg', 'emp', 'cat']

Hier lesen wir jetzt die Datei mit ihren Spalten ein:

In [ ]:
dataset = pandas.read_csv(input_data, sep='\t', names=names)

Im nächsten Schritt "erklären" wir die Features in der Datei: Die 
erste Spalte ist der Text (den wir ignorieren), danach kommen drei 
Zahlenwerte, die für die Berechnung genutzt werden, in der letzten 
Spalte steht die Kategorie, die wir klassifizieren wollen:

In [ ]:
array = dataset.values

X = array[:,1:4]

X = X.astype('int')

Y = array[:,4]

Y = Y.astype('str')

Dann teilen wir den Korpus in 80% Trainings- und 20% Testdaten 
zufällig auf:

In [ ]:
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

Es gibt verschiedene Klassifikatoren, die man jetzt ausprobieren könnte. 
Wir entscheiden uns der Einfachheit halber für einen, den Decision Tree Classifier, und 
trainieren unser Modell damit:

In [ ]:
def train_model():
    classifier = DecisionTreeClassifier()
    classifier.fit(X_train,Y_train)
    model_file = 'finalized_model.sav'
    pickle.dump(classifier, open(model_file, 'wb'))
    
train_model()

Das Modell ist nun in einer Datei mit dem Namen 'finalized_model.sav' 
gespeichert und wir können es anwenden. Wenn wir es auf einen Text 
anwenden wollen, dann müssen wir diesem Text dieselben Werte zuweisen 
wie den Trainingstexten, also Polarität nach Wortlistenvergleich, Anzahl 
der Negationen und Anzahl der Verstärker. 


In [ ]:
def pred_senti_sentence(sent):
    (neg, emp) = neg_emp_in_sentence(sent)
    polarity = sentiment_analysis(sent)
    sent_array = [[polarity,neg,emp]]
    model = pickle.load(open('finalized_model.sav', 'rb'))
    result = model.predict(sent_array)[0]
    return(result)

Jetzt kann man Texte prüfen und ein Gefühl dafür bekommen, wie gut das 
Modell ist:

In [ ]:
pred_senti_sentence("Wir leiden unter Bahnlärm")

In [ ]:
 pred_senti_sentence("Meine Strecke war stundenlang gesperrt")

In [ ]:
pred_senti_sentence("Bei der Bahn ist WLAN kostenlos!")

Die Qualität der Klassifikation hängt einerseits von der Qualität der Merkmale 
 und andererseits von der Größe und der Ausgewogenheit des 
Trainingskorpus ab. 

Mit den Daten der GermEval stellen wir  fest, dass die Klassifikation von positiven Texten nicht gelingt. Egal, wie das Modell trainiert ist, kein Satz wird als positiv klassifiziert.
Ein näherer Blick in die Daten zeigt, warum das so ist: 590 negative und 1199 neutrale Texte stehen lediglich 151 positiven gegenüber. Wir haben also einen nicht ausgewogenen Datensatz. Die Gesamt-Accuracy ist damit einfach am höchsten, wenn das Modell annimmt, dass es keine positiven Meinungsäußerungen im Datensatz gibt.
Wenn man nun mit einem ausgewogenen Datensatz trainiert, also mit je ca. 150 negativen, neutralen und positiven Texten, dann gelingt auch die Klassifikation mit dem trainierten Modell. Allerdings ist der Datensatz dann letztlich zu klein, um ein wirklich gutes Modell trainieren zu können. Man könnte sich auch vorstellen, die 151 positiven Texte im Datensatz zu erweitern, indem man sie kopiert und leicht modifiziert, also Varianten davon erzeugt. Das Problem des maschinellen Lernens auf nicht ausgewogenen Datensätzen und verschiedene Methoden, damit umzugehen, werden  bei Haixiang et al. (2017) beschrieben. 

Forschungsgruppen haben mit weiteren Merkmalen experimentiert, wie der Zahl der Sentiment-Wörter im Text, dem maximalen Polaritätswert im Text, den negativen und den positiven Polaritätswerten, dem Polaritätswert des letzten Sentiment-Wortes im Satz oder den Tf-idf-Werten für alle Wörter im Text. 

(Beim Tf-idf-Maß handelt es sich um ein Maß aus dem Information Retrieval, bei dem die Häufigkeit des Auftretens eines Wortes in einem Dokument mit der Häufigkeit des Auftretens in der gesamten Dokumentmenge in  Beziehung gesetzt wird. Für nähere Informationen dazu siehe Ferber (2003).)
